## SAGESim Implementation: SIR Model Example

We demonstrate how to implement an **Agent-Based Model (ABM)** of the **Susceptible-Infected-Recovered (SIR)** model using the **sagesim** library. **sagesim** is a powerful tool for large-scale agent-based simulations, utilizing parallel computing to model complex systems such as disease spread across large populations. By the end of this notebook, you will not only have a fully functional **SIR-ABM simulator** that illustrates how individual agent behaviors influence disease transmission dynamics, but also gain valuable insights into how the **sagesim** library works, enabling you to effectively leverage this tool for your own tasks.

---

### **ABM-SIR Model**

The **SIR model** is a fundamental framework in epidemiology for understanding how infectious diseases spread through a population. By employing **ABM**, we can simulate disease dynamics at the individual level, capturing the complexity of interactions between individuals and how these interactions drive disease transmission.

In this notebook, we focus on an ABM implementation of the SIR model, where each **agent** represents an individual with unique characteristics—specifically, their **preventative measures**. These preventative measures influence how each agent responds to exposure and ultimately affect the spread of disease throughout the population. Each agent can be in one of the following three states:

- **Susceptible**: The agent is healthy but vulnerable to infection upon contact with an infected agent.
- **Infected**: The agent is currently carrying the disease and can transmit it to susceptible agents through interactions.
- **Recovered**: The agent has recovered from the infection and is assumed to be immune, meaning they are no longer at risk of reinfection.

Disease transmission occurs through interactions between agents. Importantly, the probability of a susceptible agent becoming infected is **not uniform**—it depends on their individual preventative behaviors. These behaviors, encoded as each agent's internal **preventative measures**, reflect real-world health practices such as:

- Hygiene practices (e.g., handwashing, mask-wearing)
- Social distancing
- Vaccination status

Agents with stronger preventative measures are **less likely** to become infected, even when exposed to the virus. This heterogeneity introduces realism to the model, allowing us to study how individual differences in behavior impact the overall dynamics of disease spread.

---

### **`sagesim` Implementation**

In this notebook, we will walk you through the detailed process of creating and simulating a **SIR-ABM** using the **sagesim** library. The modeling process involves defining two key functions and implementing two core Python classes:

1. **`step_func()`**  
   This function specifies the behavior of agents in a particular breed over time. It determines how agents transition between states—such as moving from **Susceptible** to **Infected**—based on interactions and predefined rules, essentially defining the agent's behavior at each time step.

2. **`reduce_agent_data_tensors_()`**  
   **sagesim** operates using parallel computing, where agents are distributed across multiple compute nodes or GPUs. After executing the predefined steps, the simulation generates copies of agent data across different nodes. This function handles the logical combination (or "reduction") of those copies into a single consistent version.

3. **`class SIRBreed(Breed)`**  
   This problem-specific breed class, **SIRBreed**, is a subclass of the `Breed` class in **sagesim**. It is responsible for registering the properties and the `step_func()`.

4. **`class SIRModel(Model)`**  
   The problem-specific model class, **SIRModel**, is a subclass of the `Model` class from **sagesim**. It defines the SIR-ABM model by registering the breed(s) and the reduction function, while managing the creation and connection of agents.

Once the SIRModel is defined, we can use it to create the corresponding ABM model, and simulate the spread of disease across the population.



In [ ]:
from sagesim.breed import Breed
from sagesim.model import Model
from sagesim.space import NetworkSpace

import cupy as cp


def step_func(
    agent_ids,
    agent_index,
    globals,
    breeds,
    locations,
    popularities,
    vehicle_nums,
):

    # Get agent's vehicle number and neighbors' locations
    agent_vehicle_num = vehicle_nums[agent_index]
    # Zero out agent's vehicle count
    vehicle_nums[agent_index] = agent_vehicle_num
    neighbor_ids = locations[agent_index]

    # find total popularity of all neighbors
    total_popularity = cp.float64(0)
    neighbor_i = 0
    while not cp.isnan(neighbor_ids[neighbor_i]) and neighbor_i < len(neighbor_ids):
        neighbor_id = neighbor_ids[neighbor_i]
        # Find the index of the neighbor_id in agent_ids
        neighbor_index = -1
        i = 0
        while i < len(agent_ids) and agent_ids[i] != neighbor_id:
            i += 1
        if i < len(agent_ids):
            neighbor_index = i
            neighbor_popularity = popularities[int(neighbor_index)]
            total_popularity += neighbor_popularity
        neighbor_i += 1

    remainder = agent_vehicle_num
    largest_alloc = 0

    if total_popularity > 0:
        remainder_alloc_index = -1
        neighbor_i = 0
        while not cp.isnan(neighbor_ids[neighbor_i]) and neighbor_i < len(neighbor_ids):
            neighbor_id = neighbor_ids[neighbor_i]
            # Find the index of the neighbor_id in agent_ids
            neighbor_index = 0
            while (
                neighbor_index < len(agent_ids)
                and agent_ids[neighbor_index] != neighbor_id
            ):
                neighbor_index += 1
            if (neighbor_index < len(agent_ids)) and (agent_ids[i] != neighbor_id):
                neighbor_popularity = popularities[int(neighbor_index)]

                neighbor_allocation = int(
                    agent_vehicle_num * neighbor_popularity / total_popularity
                )
                # find the top popularity neighbor
                if neighbor_allocation > largest_alloc:
                    remainder_alloc_index = neighbor_index
                    largest_alloc = neighbor_allocation

                remainder -= neighbor_allocation
            neighbor_i += 1

        # Distribute the remainder (due to rounding) to top contributors
        if remainder > 0 and remainder_alloc_index >= 0:
            vehicle_nums[int(remainder_alloc_index)] += remainder



In [ ]:

def reduce_agent_data_tensors_(adts_A, adts_B):
    result = []
    # breed would be same as first
    result.append(adts_A[0])
    # network would be same as first
    result.append(adts_A[1])
    # OSMnxid would be same as first
    result.append(adts_A[2])
    # Popularity would be samge as first
    result.append(adts_A[3])
    # Num of vehicles at this intersection would be summed
    result.append(adts_A[4] + adts_B[4])
    return result


In [ ]:

class SFRBreed(Breed):

    def __init__(self) -> None:
        name = "SFR"
        super().__init__(name)

        # register osmnxid
        # self.register_property("osmnxid")

        # popularity of the agent, real value between [0,1]
        self.register_property("popularity", default=0.5)

        # number of vehicles at the current agent/node/intersection
        self.register_property("vehicle_num", default=10)

        self.register_step_func(step_func)



In [ ]:

class SFRModel(Model):

    def __init__(self) -> None:
        space = NetworkSpace()
        super().__init__(space)
        self._sfr_breed = SFRBreed()
        self.register_breed(breed=self._sfr_breed)
        self.register_reduce_function(reduce_agent_data_tensors_)

    def create_agent(self, popularity, vehicle_num):
        agent_id = self.create_agent_of_breed(
            self._sfr_breed,
            popularity=popularity,
            vehicle_num=vehicle_num,
        )
        self.get_space().add_agent(agent_id)
        return agent_id

    def connect_agents(self, agent_0, agent_1):
        self.get_space().connect_agents(agent_0, agent_1)


## Generate Small-World Network
We use the Watts-Strogatz model to generate a small-world network.

In [ ]:
from time import time
from random import random, sample
import networkx as nx
from mpi4py import MPI


# MPI environment setup
comm = MPI.COMM_WORLD
num_workers = comm.Get_size()
worker = comm.Get_rank()

def generate_small_world_network(n, k, p):
    """
    Generate a small-world network using the Watts-Strogatz model.

    Parameters:
    - n (int): Number of nodes (agents).
    - k (int): Each node is connected to its k nearest neighbors.
    - p (float): Probability of rewiring an edge (introduces randomness).

    Returns:
    - networkx.Graph: Generated network.
    """
    return nx.watts_strogatz_graph(n, k, p)

## 👥 Create and Initialize Agents
Agents are placed in the network and randomly assigned an initial state.


In this simple SIR model, we define only **one breed**, representing the general population. 

In [13]:
def generate_small_world_of_agents(model, num_agents: int, num_init_connections: int, num_infected: int) -> SIRModel:
    """
    Create a SIR model with agents in a small-world network topology.

    Parameters:
    - model (SIRModel): An instance of the SIR model.
    - num_agents (int): Total number of agents.
    - num_init_connections (int): Each agent is connected to this many neighbors.
    - num_infected (int): Number of initially infected agents.

    Returns:
    - SIRModel: Initialized model with agents and connections.
    """
    network = generate_small_world_network(num_agents, num_init_connections, 0.2)

    for n in network.nodes:
        preventative_measures = [random() for _ in range(100)]
        model.create_agent(SIRState.SUSCEPTIBLE.value, preventative_measures)

    for n in sample(sorted(network.nodes), num_infected):
        model.set_agent_property_value(n, "state", SIRState.INFECTED.value)

    for edge in network.edges:
        model.connect_agents(edge[0], edge[1])

    return model

## ⚙️ Define Simulation Parameters
You can modify these values to run different configurations.

In [14]:
num_agents = 1000
num_init_connections = 6
num_nodes = 1  # Logical nodes, can be used for partitioning

## 🛠️ Set up the SIR Model

In [ ]:
model = SIRModel()
model.setup(use_gpu=True)  # Enables GPU acceleration if available

AttributeError: 'SIRModel' object has no attribute 'register_reduce_function'

## 🧱 Build the Agent Network

In [ ]:
model_creation_start = time()

model = generate_small_world_of_agents(
    model,
    num_agents,
    num_init_connections,
    int(0.1 * num_agents),  # 10% initially infected
)

model_creation_end = time()
model_creation_duration = model_creation_end - model_creation_start
print(f"Model creation took {model_creation_duration:.2f} seconds.")

## ▶️ Run the Simulation

In [ ]:
simulate_start = time()

model.simulate(num_ticks=10, sync_workers_every_n_ticks=1)

simulate_end = time()
simulate_duration = simulate_end - simulate_start
print(f"Simulation took {simulate_duration:.2f} seconds.")

## 📊 Collect Final States

In [ ]:
if worker == 0:
    result = [
        SIRState(model.get_agent_property_value(agent_id, property_name="state"))
        for agent_id in range(num_agents)
        if model.get_agent_property_value(agent_id, property_name="state") is not None
    ]
    print(f"Final state distribution: {[str(state) for state in result[:10]]}...")

## 💾 Save Execution Metrics

In [ ]:
import csv

if worker == 0:
    with open("execution_times.csv", "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            num_agents,
            num_init_connections,
            num_nodes,
            num_workers,
            model_creation_duration,
            simulate_duration
        ])
    print("Execution time written to 'execution_times.csv'.")